In [1]:
# Setup and import libraries
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import requests
import json
import gmaps

from config import gkey
gmaps.configure(api_key=gkey)

df = pd.read_csv("../Output/weather_data.csv")

In [2]:
#--- Ideal Conditions: ---#
#                         #
#    Temp <85, >70 F      #
#    Humidity < 50%       #
#    Windspeed < 10mph    # 
#    Cloudiness < 20%     #
#                         #
#-------------------------#

holiday_df = df[(df['Temp'] < 85) &
                (df['Temp'] > 70) &
                (df['Humidity'] < 50) &
                (df['Windspeed'] < 10) &
                (df['Cloudiness'] < 20)].copy()

holiday_df

,Name,Lat,Long,Temp,Humidity,Cloudiness,Windspeed,Country,Datetime
59,Vredendal,-31.6683,18.5012,82.04,34,7,9.91,ZA,1618905593
208,Faya,18.3851,42.4509,75.20,17,0,4.61,SA,1618905640
258,Buckeye,33.3703,-112.5838,71.01,12,0,5.26,US,1618905631
331,Alice Springs,-23.7000,133.8833,77.00,14,0,6.91,AU,1618905668
373,Tessalit,20.1986,1.0114,83.61,8,0,7.70,ML,1618905681
397,Hukuntsi,-24.0000,21.7500,71.76,26,0,7.99,BW,1618905687
444,Yulara,-25.2406,130.9889,78.80,17,0,5.75,AU,1618905698
531,Bundaberg,-24.8500,152.3500,77.00,43,0,9.22,AU,1618905719


In [3]:
# API REQUESTS FOR HOTEL DATA

# Add empty columns to holiday_df to store hotel details  
holiday_df['Nearest Hotel'] = ""
holiday_df['Hotel Distance'] = ""

# Set base url and parameters dictionary for places API request
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "keyword": "Hotel",
    "radius": 5000,
    "type": "lodging",
    "key": gkey
}

# Set base url for the hotel distance from city centre request 
distance_url = "https://maps.googleapis.com/maps/api/distancematrix/json"
params2 = {"key": gkey,
          "units" : "metric"}

# Create a list to store any cities without hotels within 5000m
no_close_hotels = ["There is great weather in"]

# Use iterrows to iterate through the dataframe, adding each cities hotel data
for index, row in holiday_df.iterrows():

    lat = row['Lat']
    long = row["Long"]
    params["location"] = origin = f"{lat},{long}"
    # origin = f"{lat},{long}"

    try: 
        # call the API and add hotel name to dataframe. 
        response = requests.get(base_url, params).json()
        holiday_df.loc[index, 'Nearest Hotel'] = response['results'][0]['name']
        place_id = response['results'][0]['place_id']
      
        # calculate distance from city centre with distance matrix API request
        params2["origins"] = f"{lat},{long}"
        params2["destinations"] = f"place_id:{place_id}"                                           
      
        response = requests.get(distance_url, params2).json()
        distance = response["rows"][0]["elements"][0]["distance"]["text"]
        holiday_df.loc[index, 'Hotel Distance'] = distance
    except: 
        no_close_hotels.append(row['Name'])

# Print the names of any cities without hotels within 5000m
if len(no_close_hotels) > 0:
    no_close_hotels = [hotel+"," for hotel in no_close_hotels]
    no_close_hotels.append("but no hotels within 5000m found on search")
    print(*no_close_hotels)
    
holiday_df

There is great weather in, Faya, Tessalit, but no hotels within 5000m found on search


,Name,Lat,Long,Temp,Humidity,Cloudiness,Windspeed,Country,Datetime,Nearest Hotel,Hotel Distance
59,Vredendal,-31.6683,18.5012,82.04,34,7,9.91,ZA,1618905593,River Lodge,1.3 km
208,Faya,18.3851,42.4509,75.20,17,0,4.61,SA,1618905640,,
258,Buckeye,33.3703,-112.5838,71.01,12,0,5.26,US,1618905631,Buckeye Motor Hotel,0.8 km
331,Alice Springs,-23.7000,133.8833,77.00,14,0,6.91,AU,1618905668,DoubleTree by Hilton Hotel Alice Springs,1.9 km
373,Tessalit,20.1986,1.0114,83.61,8,0,7.70,ML,1618905681,,
397,Hukuntsi,-24.0000,21.7500,71.76,26,0,7.99,BW,1618905687,Entabeni Hotel,4.9 km
444,Yulara,-25.2406,130.9889,78.80,17,0,5.75,AU,1618905698,Sails in the Desert,1.0 km
531,Bundaberg,-24.8500,152.3500,77.00,43,0,9.22,AU,1618905719,Burnett Riverside Hotel,2.7 km


In [4]:
# Determine the geographical centre of the dataset
# find the corners of rectangle that bounds all data points
sw = df['Lat'].min(), df['Long'].min()
se = df['Lat'].min(), df['Long'].max()
nw = df['Lat'].max(), df['Long'].min()
ne = df['Lat'].max(), df['Long'].max()

# Use the mid point of lat/long values to define the centre of the map
centre_lat = sw[0] + ((nw[0]-sw[0])/2)
centre_long = nw[1] + ((ne[1]-nw[1])/2)
centre = centre_lat, centre_long

# Map formatting 
figure_layout = {
    'width': '1000px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

# Draw the map 
fig = gmaps.figure(layout=figure_layout, center = centre, zoom_level=2)

In [5]:
# ADD HEATMAP LAYER
# Create a heatmap layer using the humidity data
locations = df[["Lat", "Long"]].astype(float)
humidity = df["Humidity"].astype(float)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=humidity.max(),
                                 point_radius = 3)

# Add to map  
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

In [6]:
# ADD MARKER LAYER
# Remove any cities without hotels within 5km 
hotel_marker_list = holiday_df.loc[holiday_df["Nearest Hotel"] != ""]

# Create lists containing city coordinates and city info 
marker_coords = []  # for marker location
hotel_name = []     # for hover text
hotel_info = []     # for info box content 

for row in range(len(hotel_marker_list)):
    marker_coords.append((hotel_marker_list.iloc[row, 1], hotel_marker_list.iloc[row, 2]))
    hotel_name.append(hotel_marker_list.iloc[row, 9])
    hotel_info.append({'city': hotel_marker_list.iloc[row, 0], 
                     'hotel': hotel_marker_list.iloc[row, 9], 
                     'distance': hotel_marker_list.iloc[row, 10], 
                     'country': hotel_marker_list.iloc[row, 7]})

# Set up the info box
info_box_template = """
<dl>
<dt>City</dt><dd>{city}</dd>
<dt>Nearest Hotel</dt><dd>{hotel}</dd>
<dt>Distance from Centre</dt><dd>{distance}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
hotel_data = [info_box_template.format(**hotel) for hotel in hotel_info]

# Add to map 
markers = gmaps.marker_layer(marker_coords, info_box_content=hotel_data, hover_text=hotel_name)


fig.add_layer(markers)
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…